In [1]:
import sys, os
import imp
import time

sys.path.insert(0,"D:/My_python_libs/qsweepy")

from numpy import *
from importlib import reload
from matplotlib.pyplot import *

import qsweepy.libraries.instruments as instruments

Failed loading module Keysight_M3102A:  No module named 'keysightSD1'
Failed loading module Keysight_M3202A:  No module named 'keysightSD1'
Failed loading module Labbrick:  [WinError 126] Не найден указанный модуль
Failed loading module ziUHF:  cannot import name 'ZIDevice' from 'qsweepy.instrument_drivers.zihdawg' (D:/My_python_libs/qsweepy\qsweepy\instrument_drivers\zihdawg.py)


In [2]:
from qsweepy.instrument_drivers._QubitDAQ.reg_intf import *
from qsweepy.instrument_drivers._QubitDAQ.usb_intf import *
import qsweepy.instrument_drivers._QubitDAQ.driver as drv
imp.reload(drv)

<module 'qsweepy.instrument_drivers._QubitDAQ.driver' from 'D:/My_python_libs/qsweepy\\qsweepy\\instrument_drivers\\_QubitDAQ\\driver.py'>

In [3]:
firmware_path = "D:/FPGA_source/qubit_daq_signaltap_tst/qubit_daq.rbf"

In [7]:
try: del adc
except: pass    
adc = drv.Device("0009052001481708", fpga_config = True, adc_config = True, firmware = firmware_path)

ADS54J40 already programmed


In [100]:
adc.debug_print = True

In [6]:
adc.usb_reset()

In [9]:
firmware_data = open(firmware_path, 'rb').read()

In [205]:
adc.print_fx3_regs()

FX3_CTRL_START         : False
FX3_CTRL_WR            : False
FX3_CTRL_ABORT         : False
FX3_CTRL_DDR_WR_BUSY   : False
FX3_CTRL_DDR_RD_BUSY   : False
FX3_CTRL_RESET         : False
FX3_CTRL_PATH          : True
FX3_CTRL_ONCHIP_WR_BUSY: False
FX3_CTRL_ONCHIP_RD_BUSY: False

FX3_LEN:    16384
FX3_ADDR:   131072
FX3_CRC32:  0x4776c054
FX3_FIFO_LVL_DDR_WR: 0
FX3_FIFO_LVL_DDR_RD: 0


In [6]:
s = b'\xcf\xf0\xe8\xf1\xee\xe5\xe4\xe8\xed\xe5\xed\xed\xee\xe5 \xea \xf1\xe8\xf1\xf2\xe5\xec\xe5 \xf3\xf1\xf2\xf0\xee\xe9\xf1\xf2\xe2\xee \xed\xe5 \xf0\xe0\xe1\xee\xf2\xe0\xe5\xf2.\r\n\n'
str(s, "windows-1251")

'Присоединенное к системе устройство не работает.\r\n\n'

In [114]:
#Abort
adc.write_reg(FX3_BASE, FX3_CTRL, FX3_CTRL_ABORT)

In [121]:
#Reset
adc.write_reg(FX3_BASE, FX3_CTRL, FX3_CTRL_RESET)

In [8]:
adc.i2c_reset()

In [13]:
adc.get_trig_clk_phase_shift()

6.597800925925925

## DDR3 LOOPBACK

In [30]:
data_len = 50000000
print(data_len,"32-bit words")

print(data_len*4/(1024**2), "Mb")
print(data_len*4, "b")

50000000 32-bit words
190.73486328125 Mb
200000000 b


In [31]:
sample_width = 32
rng = random.default_rng()
data = rng.integers(-2**sample_width/2+1,2**sample_width/2, data_len, dtype = int)

In [32]:
#Write
t1 = time.time()
bytes_tx = adc.write_ddr3(0, data)
t2 = time.time()
print("Transferrd {:d}".format(bytes_tx))
print("Time: {:f}s".format(t2-t1))
print("Speed: {:f}Mbytes/s".format( (data_len*4/(1024**2))/(t2-t1) ) )

Transferrd 200000000
Time: 1.627591s
Speed: 117.188438Mbytes/s


In [33]:
#Read
t1 = time.time()
rd_data = adc.read_ddr3(0,data_len,int)
t2 = time.time()
print("Transferrd {:d}".format(bytes_tx))
print("Time: {:f}s".format(t2-t1))
print("Speed: {:f}Mbytes/s".format( (data_len*4/(1024**2))/(t2-t1) ) )

Transferrd 200000000
Time: 0.998982s
Speed: 190.929284Mbytes/s


In [59]:
len(rd_data)

123

In [60]:
%matplotlib
#plot(rd_data-data)
plot(rd_data)
plot(data)

Using matplotlib backend: Qt5Agg


## Onchip memory loopback

In [157]:
data_len = 8192*4 #1.5*(1024*16)/4
print(data_len,"32-bit words")
print(data_len*4/(1024**2), "Mb")
print(data_len*4, "b")

32768 32-bit words
0.125 Mb
131072 b


In [164]:
sample_width = 32
rng = random.default_rng()
data = rng.integers(-2**sample_width/2+1,2**sample_width/2, int(data_len), dtype = int)

In [165]:
#Write
adc.write_feature_memory(0, data)

131072

In [166]:
#Read
rd_data = adc.read_feature_memory(0, data_len, dtype = int)

In [167]:
%matplotlib
#plot(rd_data-data)
plot(rd_data)
plot(data)

Using matplotlib backend: Qt5Agg


## Data capture

In [5]:
adc.nsamp = int(8192)
adc.nsegm = 30
adc.trig_mode = "ext"
adc.set_trig_src_period(10000)
adc.set_trig_src_width( 200)
adc.start_wait_done()

True

In [5]:
adc.nsamp = 8192
adc.nsegm = 10
adc.trig_mode = "man"
adc.set_trig_src_period(10000)
adc.set_trig_src_width( 200)
adc.start()
while(1):
    if adc.check_done():
        break
    if adc.check_trig_ready():
        adc.soft_trig()

In [9]:
adc.set_trig_delay(200e-9)

In [9]:
adc.print_pp_regs()

START         : False
BUSY          : False
ABORT         : False
READY         : False
EXT_TRIG_EN   : True
TRIG_EDGE     : False
SOFT_TRIG     : False
RES_DMA_BUSY  : False
RES_DMA_READY : True

CAP_LEN          : 1024
CAP_FIFO_LVL     : 40
NSEGM            : 3
TRIG_DLY         : 0
RES_DMA_FIFO_LVL : 2


In [169]:
#Read
rd_data = adc.get_data()

In [165]:
%matplotlib
plot(rd_data.real.T)
plot(rd_data.imag.T)

Using matplotlib backend: Qt5Agg


In [ ]:
aver_data = adc.get_averaged_data()

In [192]:
data = adc.read_feature_memory( 0x20000 , 2*adc.nsamp, dtype = int32)
for d in data:
    print( hex(d&0xFFFFFFFF)," ", end="")

0x1ffff  0x1ffff  0x1fffd  0x3  0x1fffd  0x10003  0xffff  0x6  0x1fff8  0x1fffa  0x1fffe  0x1ffff  0x10000  0x2fffa  0x1fffc  0x10002  0x1fffd  0x1fffc  0x2  0x1fffd  0x4  0xffff  0x1fffc  0xfffe  0x10000  0x1ffff  0x1fffe  0x10001  0x10000  0xfffe  0x2fffc  0x2  0x1fffd  0x2fffb  0x2fffb  0x10001  0x4  0x10000  0x3  0x1fffd  0xffff  0xfffc  0x1fffc  0x10002  0xffff  0x10003  0x1ffff  0x6  0x1fffc  0x2  0x1fffc  0x4  0x10000  0x1fffa  0x10001  0xffff  0x10002  0x10002  0x1fffc  0x3  0xffff  0x1fffd  0x2fffb  0x10000  0x3  0x2fffa  0x2fffa  0xfffe  0x1fffb  0xffff  0xfffe  0xffff  0xfffe  0x10001  0xfffe  0x7  0x10001  0x10000  0x3  0x3  0x20000  0x1fffd  0x1fffb  0xffff  0x1fffb  0xfffe  0x8  0x1fffd  0x10002  0x2fff9  0x1fffe  0x10001  0xffff  0x1fffe  0xffff  0x10002  0x10000  0x10001  0x1fffc  0x10001  0x1fffd  0x1fffc  0x10001  0x10003  0xffff  0x1fffd  0x10000  0x10001  0x10000  0x10000  0xfffc  0x1fffd  0x10000  0x10000  0x5  0x10000  0x10002  0x10000  0x2fffc  0x1fffe  0x1fffe  

0x237  0x3  0x233  0x1fffd  0x225  0x1fffc  0x223  0x10000  0x224  0x3  0x21a  0xffff  0x21d  0x2fffb  0x20f  0x1fffe  0x207  0x2  0x206  0xfffd  0x203  0x10003  0x1f7  0x4  0x1f6  0x3  0x1ea  0x1ffff  0x1ea  0x3  0x1de  0x1fffd  0x1d9  0x4  0x1d6  0x10002  0x1cd  0xffff  0x1c8  0xfffe  0x1c7  0x10000  0x1c0  0x4  0x1b9  0x10000  0x1b3  0x10001  0x1ac  0x6  0x1aa  0x6  0x1a8  0x1fffe  0x199  0x1fffe  0x194  0x10001  0x191  0xffff  0x18a  0x1fffd  0x184  0x1fffc  0x180  0x3  0x183  0x10000  0x17b  0x2fffb  0x16b  0x1fffc  0x168  0x10001  0x168  0x4  0x161  0x10001  0x157  0x10000  0x156  0x10002  0x151  0x10001  0x14d  0x1fffe  0x149  0x1fffd  0x13a  0x1ffff  0x13f  0x1fffe  0x13a  0x1  0x132  0x1fffc  0x12c  0x10001  0x127  0x1fffc  0x126  0x2fffa  0x11d  0x2fffc  0x11b  0x6  0x118  0x1  0x10c  0x10001  0x103  0x1fffd  0x104  0x2  0x101  0x1fffd  0xf9  0x1fffd  0xf4  0x20000  0xef  0x10002  0xef  0x10001  0xe7  0x20002  0xdb  0x2fffd  0xd8  0x10002  0xdd  0x10001  0xd3  0x2fffa  0xca  

 0x2fb17  0x3  0x2fb10  0x2fffb  0x2fb13  0x1fffc  0x2fb17  0x1fffd  0x2fb13  0x10000  0x2fb19  0x10001  0x2fb18  0x10000  0x2fb15  0xffff  0x2fb13  0x1fffa  0x2fb12  0x1fffb  0x2fb13  0x3  0x2fb0f  0x10000  0x2fb13  0xfffd  0x2fb0f  0x1ffff  0x2fb12  0x4  0x2fb10  0x2fff4  0x2fb0e  0x1fffe  0x2fb12  0x10000  0x2fb15  0xffff  0x2fb0d  0x10000  0x2fb12  0x1fffc  0x2fb10  0x10001  0x2fb0e  0x1fffc  0x2fb0e  0x1ffff  0x2fb0e  0x10000  0x2fb0a  0x2fffa  0x2fb10  0xffff  0x2fb11  0x10000  0x2fb0b  0x10001  0x2fb0f  0x1fffc  0x2fb0d  0xfffe  0x2fb10  0x1fffe  0x2fb0d  0xffff  0x2fb09  0x1fffa  0x2fb0e  0x4  0x2fb09  0x1fffd  0x2fb0d  0x1fffc  0x2fb07  0xffff  0x2fb0d  0x10001  0x2fb08  0x2  0x2fb05  0x1fffd  0x2fb07  0x4  0x2fb0d  0x4  0x2fb0d  0xffff  0x2fb08  0xfffc  0x2fb05  0x3  0x2fb08  0x1ffff  0x2fb0b  0x1ffff  0x2fb08  0x10000  0x2fb01  0xfffe  0x2fb02  0x10003  0x2fb03  0x1fffc  0x2fb06  0x1fffb  0x2fb04  0xffff  0x2fb02  0x20000  0x2fb01  0x1fffa  0x2fb04  0x2fffc  0x2fb02  0x1fffc

  0x2fa3b  0x5  0x2fa42  0xffff  0x2fa47  0x2  0x2fa48  0x10000  0x2fa52  0x1fffe  0x2fa57  0x10002  0x2fa59  0x1  0x2fa65  0x1fffe  0x2fa6a  0xfffe  0x2fa6b  0x4  0x2fa6f  0xfffd  0x2fa6d  0x10000  0x2fa7c  0x10000  0x2fa7e  0x1fffe  0x2fa84  0x10003  0x2fa8e  0xfffe  0x2fa8e  0x2  0x2fa95  0x1  0x2fa97  0x1fffd  0x2faa1  0x10000  0x2faa4  0x10000  0x2faa9  0x1fffd  0x2faad  0xffff  0x2fab6  0x1fffd  0x2fab9  0x1ffff  0x2fac3  0x1fffe  0x2fac3  0x1fffd  0x2fac5  0x20000  0x2faca  0x2  0x2fad1  0x1  0x2fad9  0x2fffb  0x2fada  0xffff  0x2fae5  0x3  0x2fae8  0x1fffd  0x2fae4  0x1fffc  0x2faee  0xffff  0x2faf7  0x1fffd  0x2faf3  0x2fffc  0x2fafe  0xfffe  0x2fb04  0x1fffe  0x2fb06  0x2fffa  0x2fb0d  0x1fffc  0x2fb0e  0x1fff8  0x2fb10  0x10000  0x2fb1b  0x1fffb  0x2fb22  0x2  0x2fb25  0x1fffb  0x2fb25  0x4  0x2fb35  0x5  0x2fb31  0x2fffd  0x2fb35  0x1fffa  0x2fb3a  0xfffe  0x2fb41  0x20000  0x2fb46  0xffff  0x2fb46  0xfffd  0x2fb4f  0x1fffc  0x2fb52  0x2  0x2fb54  0x1fffe  0x2fb58  0x2fffb 

0x1fffb  0x1cc  0x4  0x1c4  0x1fffc  0x1cd  0x1fffd  0x1c8  0x1fffa  0x1c8  0x10001  0x1ce  0x2fffa  0x1cc  0x2fffb  0x1ca  0x2fff8  0x1c9  0xffff  0x1d0  0x1fffa  0x1d1  0x2fffc  0x1d2  0x10001  0x1cb  0x10003  0x1d3  0x10001  0x1d3  0x2fff9  0x1ce  0x1fffa  0x1d1  0xfffc  0x1d1  0x10002  0x1d9  0x1fffd  0x1d1  0x4  0x1d1  0x1fffe  0x1d1  0x1fffb  0x1d5  0x2fffd  0x1d7  0x1fff9  0x1d5  0x1  0x1da  0x1fffd  0x1da  0x2fffa  0x1d7  0xfffe  0x1d4  0x5  0x1da  0x10000  0x1e0  0xffff  0x1dc  0x2fffa  0x1d8  0x1  0x1da  0x10003  0x1d6  0x2fff7  0x1db  0xffff  0x1df  0x20000  0x1db  0x4  0x1db  0x1fffc  0x1db  0xfffd  0x1d6  0x10001  0x1dc  0x3  0x1e0  0x1fffe  0x1db  0x1fffa  0x1df  0x1ffff  0x1e6  0x4  0x1e3  0x1fffe  0x1de  0x1ffff  0x1df  0x1fffe  0x1e5  0x10000  0x1e7  0x10001  0x1e4  0x1fffc  0x1e3  0x1fffe  0x1e6  0x5  0x1e3  0x10000  0x1e4  0x10000  0x1e1  0x10000  0x1dc  0x10000  0x1e5  0x2  0x1e6  0x4  0x1e9  0x5  0x1e7  0xfffe  0x1e7  0xffff  0x1ec  0x2fff9  0x1e7  0x3  0x1e8  0x10

0x10000  0x1ffff  0x3  0x1fffc  0x10002  0x8  0x1ffff  0x2fffb  0x10000  0x2fffc  0x20000  0x20001  0x10002  0x5  0x10006  0x10000  0xffff  0xffff  0x2fffa  0x1fffe  0x5  0x1fffd  0x10000  0x1fffd  0x1fffd  0x2  0x20000  0x1ffff  0x10000  0x20000  0xfffe  0x1fffb  0x2fff9  0x10005  0x1  0x2fff9  0x1fffd  0x1fffc  0x10000  0x1ffff  0xfffd  0x1fffd  0x2fffc  0x10002  0x10000  0xfffe  0xfffe  0x2fff6  0x10000  0x2fffb  0x1fffe  0x1ffff  0x2  0x10003  0x1fffd  0x1fffe  0x3  0x10000  0x1fffd  0x10001  0x20000  0x10001  0xffff  0x1fffe  0x10001  0x10003  0x10000  0xfffe  0x1fffd  0x3  0xfffe  0x1fffe  0x10001  0x1fff9  0x10000  0xffff  0x4  0x2fff8  0x10001  0x1fffc  0x2fff9  0x1ffff  0x1ffff  0x1fffe  0x10000  0x1fffd  0x5  0x10000  0x2fffb  0x3  0x2  0x1fffc  0x10000  0x5  0x1fffe  0x2  0x2  0x2fffd  0x1fffe  0x1fffc  0x10001  0x5  0x1fffc  0x10000  0xfffe  0x10000  0x10002  0x10000  0x20000  0xfffe  0x1fffc  0x1fff8  0x1fffc  0x2  0x10000  0xfffe  0x3  0x1ffff  0x2fffa  0xfffe  0x2fffa  0

 0x1fffc  0x2ff4b  0xfffd  0x2ff41  0x10001  0x2ff41  0x10000  0x2ff3a  0x1fffc  0x2ff3a  0x1ffff  0x2ff35  0x10000  0x2ff2f  0xfffe  0x2ff2d  0x1fff8  0x2ff28  0x2fff9  0x2ff27  0x4  0x2ff27  0x3  0x2ff23  0x10001  0x2ff21  0x10000  0x2ff15  0x2fffb  0x2ff13  0x3  0x2ff12  0x10002  0x2ff10  0x2fff8  0x2ff04  0x10002  0x2ff03  0x2fffd  0x2ff01  0x3  0x2fef5  0x2fff6  0x2fef6  0x20000  0x2fef8  0x10002  0x2fef2  0xfffc  0x2feeb  0x2fff9  0x2fee8  0x10004  0x2fee9  0x3  0x2fee0  0x10001  0x2fee4  0xfffc  0x2fedc  0x1fffe  0x2fed9  0x10001  0x2fed3  0x1fffd  0x2fecf  0x1fffb  0x2fec9  0x10001  0x2fec0  0xfffe  0x2fec8  0x3  0x2fec2  0x1fffe  0x2febf  0x2  0x2fec2  0x2fffa  0x2febb  0x20000  0x2feb6  0xffff  0x2fead  0x10001  0x2feaf  0x10001  0x2feab  0x3  0x2fea3  0x1  0x2fea2  0xfffe  0x2fe9d  0xfffd  0x2fea0  0x1fffe  0x2fe9b  0x1fffc  0x2fe92  0xfffc  0x2fe8e  0x1fffb  0x2fe90  0x1fffe  0x2fe8d  0xffff  0x2fe84  0x10001  0x2fe84  0x1fffa  0x2fe88  0x5  0x2fe7d  0x4  0x2fe7a  0x20000  

  0x3  0x2ea98  0x10002  0x2eaa1  0x1fffd  0x2eab0  0x1  0x2eabe  0x5  0x2eacf  0x2fffc  0x2ead5  0x1fffe  0x2eae2  0x1fffe  0x2eafa  0x1fffc  0x2eafc  0x1fffb  0x2eb0b  0x2  0x2eb17  0x5  0x2eb22  0x1  0x2eb31  0x1fffd  0x2eb35  0xfffe  0x2eb49  0x10001  0x2eb57  0x10000  0x2eb66  0x10001  0x2eb69  0x2fffa  0x2eb7c  0x5  0x2eb88  0x1fffd  0x2eb96  0x10002  0x2eb9d  0xfffe  0x2ebad  0x1fffd  0x2ebb4  0x10001  0x2ebc2  0xfffe  0x2ebcb  0x1fffe  0x2ebde  0x1ffff  0x2ebea  0x10000  0x2ebf1  0xffff  0x2ebff  0x1fffd  0x2ec0b  0x20001  0x2ec16  0x1fffb  0x2ec2b  0x2fffc  0x2ec31  0x1fffe  0x2ec39  0x1ffff  0x2ec45  0x10000  0x2ec55  0x1fffa  0x2ec5b  0xfffe  0x2ec6b  0x10001  0x2ec7c  0x1fffe  0x2ec7c  0x1fffe  0x2ec8d  0xfffe  0x2ec9a  0x1ffff  0x2ecab  0x1fffc  0x2ecb8  0x1fffc  0x2ecb9  0xfffd  0x2ecc2  0x10003  0x2ecd4  0x10002  0x2ecdd  0x1fffb  0x2ece7  0x1fffd  0x2ecf1  0x1  0x2ed03  0x1fffd  0x2ed07  0x1fffb  0x2ed17  0x2fffc  0x2ed1e  0x1fffc  0x2ed2c  0x1fffa  0x2ed39  0x10000  0x

0x2ff95  0x10000  0x2ff9f  0x1ffff  0x2ffa7  0x1fffc  0x2ffa3  0x2fffa  0x2ff9f  0x2  0x2ffa2  0xffff  0x2ffa2  0x2fffc  0x2ffa9  0x10001  0x2ffa6  0x1fffd  0x2ffad  0x10002  0x2ffad  0x10002  0x2ffb4  0xffff  0x2ffaf  0x1fffe  0x2ffb8  0xffff  0x2ffb4  0x2fffa  0x2ffbc  0xfffe  0x2ffbf  0x1ffff  0x2ffb8  0xfffd  0x2ffc2  0xffff  0x2ffc2  0x2fffa  0x2ffc4  0x10000  0x2ffc3  0x1fffe  0x2ffc8  0x2fff8  0x2ffc4  0x10000  0x2ffc4  0x0  0x2ffd0  0x10000  0x2ffc9  0x2fffa  0x2ffd0  0x10000  0x2ffd1  0x1fffc  0x2ffd7  0x1fffc  0x2ffdc  0xffff  0x2ffd5  0x2fff5  0x2ffdc  0xffff  0x2ffe0  0x2fffc  0x2ffde  0x1fffc  0x2ffde  0x1fffc  0x2ffe0  0xffff  0x2ffde  0x20000  0x2ffe2  0x1fffe  0x2ffe5  0x1fffe  0x2ffe5  0x3  0x2ffe4  0x0  0x2ffe9  0xfffd  0x2ffef  0x1fffb  0x2ffef  0x1fffe  0x2fff1  0x1fffe  0x2fff6  0x1fffd  0x2fff3  0x2fffb  0x2ffee  0x10002  0x2fff8  0x1fffb  0xfffd  0x0  0x2fff9  0x1fffe  0x1fffa  0x1fffc  0x10003  0x3  0x4  0x2fffb  0x5  0x2  0xffff  0x1  0xc  0xfffd  0xc  0xffff  

0x24e  0x2fffb  0x249  0x2  0x24d  0x1fffa  0x249  0x1ffff  0x24a  0x10002  0x24c  0x0  0x252  0x10001  0x24e  0x1fff8  0x24d  0x4  0x24e  0xffff  0x251  0x1fffd  0x24d  0xffff  0x24b  0x1fffe  0x251  0x1fffc  0x251  0x1fffe  0x24e  0x1fffd  0x24d  0x1fffe  0x24b  0x10001  0x24c  0xfffd  0x252  0x1ffff  0x24e  0x4  0x24c  0x10000  0x255  0x2fffa  0x24c  0x1fffe  0x251  0xfffe  0x248  0x10001  0x24c  0x2fffc  0x24c  0x5  0x24b  0x10002  0x252  0x20000  0x24a  0x1  0x249  0x1fffc  0x24c  0x0  0x24c  0xfffe  0x250  0x1fffd  0x24c  0x10000  0x249  0x10001  0x24d  0x10000  0x251  0x1fffa  0x24c  0x1fffe  0x24c  0xffff  0x250  0x20000  0x255  0xfffe  0x249  0x1fffe  0x24b  0xfffe  0x24f  0xfffe  0x252  0x1fffd  0x24c  0x1fffb  0x24c  0x3  0x256  0x1  0x251  0x2fffb  0x249  0xffff  0x24d  0x1ffff  0x24f  0x1ffff  0x254  0x2  0x24c  0x10006  0x251  0x10003  0x250  0x1fffb  0x250  0x3  0x24a  0x1fffa  0x24f  0x0  0x253  0x20000  0x258  0x10001  0x24d  0x1fffe  0x24f  0xffff  0x24e  0x1fffe  0x2

In [194]:
plot(data[1::2])

In [209]:
def get_averaged_data():
    data = adc.read_feature_memory( 0x20000 , 2*adc.nsamp, dtype = int32)
    data = np.reshape(data, (adc.nsamp, 2))
    return data.T[0]  + 1j * data.T[1]
data = get_averaged_data()
plot(data.real)
plot(data.imag)

In [92]:
data = adc.read_feature_memory( 0x0 , 10)

In [170]:
feature = rd_data[0]

In [171]:
adc.set_feature(feature, 0)
adc.set_feature(feature, 1)
adc.set_feature(feature, 2)
adc.set_feature(feature, 3)

In [147]:
adc.write_feature_memory(0, zeros(8192*4))

262144

In [12]:
plot(feature.real,label = "re" )
plot(feature.imag,label = "im" )
legend()

NameError: name 'feature' is not defined

In [173]:
dot_prod = adc.get_dot_prods()
print("Dot products:\n", dot_prod)
adc.threshold[0]=6945464552
adc.threshold[1]=6945464552-1
adc.threshold[2]=6945464552-2
adc.threshold[3]=6945464552-3
print("States:\n", adc.get_states())
print("Averaged dot products:")
print(adc.get_dot_prod_ave() )
print( mean(dot_prod.T[0]), mean(dot_prod.T[1]), mean(dot_prod.T[2]), mean(dot_prod.T[3]) )

def dot_calc(data, feature):
    feature_max = np.max(np.abs(feature))
    if feature_max !=0:
        feature = 2**(14-1) * feature/feature_max
    res = zeros(len(data), dtype = int64)
    for i,d in enumerate(data):
        #d=d[8:]
        res[i] = int64(sum(d.real * int16(feature.real))+ sum(d.imag * int16(feature.imag)))
    return res

dot_calc(adc.get_data(), feature)

Dot products:
 [[6954390995 6954390995 6954390995 6954390995]
 [6954235115 6954235115 6954235115 6954235115]
 [6955323571 6955323571 6955323571 6955323571]]
States:
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
Averaged dot products:
[6.95464989e+09 6.95464989e+09 6.95464989e+09 6.95464989e+09]
6954649893.666667 6954649893.666667 6954649893.666667 6954649893.666667


array([6954390995, 6954235115, 6955323571], dtype=int64)

In [12]:
a=array((1,2,3,4))
a[:2]

array([1, 2])

In [24]:
def dot_calc(data, feature):
    feature_max = np.max(np.abs(feature))
    if feature_max !=0:
        feature = 2**(14-1) * feature/feature_max
    res = zeros(len(data), dtype = int64)
    for i,d in enumerate(data):    
        res[i] = int(sum(d.real * int16(feature.real))+ sum(d.imag * int16(feature.imag)))
    return res

dot_calc(adc.get_data(), feature)

array([6936625649, 6936920365, 6937093455, 6936434909, 6936882908,
       6936658391, 6936856914, 6937022393, 6936297738, 6936165107],
      dtype=int64)

In [41]:
figure("dot_prod")
dot_prod = adc.get_dot_prods()
plot(dot_prod.T[0])

In [40]:
figure("data")
plot(rd_data[0].imag)
plot(rd_data[1].imag)
plot(rd_data[8000].imag)

In [22]:
adc.get_states()

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       ...,
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [ ]:
adc.write_feature_memory(0, ):

In [416]:
asarray(array((3,4,5,6))>3, dtype = int)

array([0, 1, 1, 1])

In [190]:
data = adc.get_data()

Time of data transfer 0.002973794937133789


In [191]:
int(ceil(64/(64*8)))*8

8

In [197]:
adc.read_ddr3(0, 8, dtype = int64)

array([-6148914691236517206], dtype=int64)

In [118]:
ceil(1.1)

2.0

In [198]:
a = array([1,2,3])

In [212]:
adc.set_threshold(-0x7123456789ABCDEF, 3)

In [219]:
adc.i2c_reset()

In [220]:
hex(adc.get_threshold(3))

USBError: [Errno None] b'libusb0-dll:err [control_msg] sending control message failed, win error: \xcf\xf0\xe8\xf1\xee\xe5\xe4\xe8\xed\xe5\xed\xed\xee\xe5 \xea \xf1\xe8\xf1\xf2\xe5\xec\xe5 \xf3\xf1\xf2\xf0\xee\xe9\xf1\xf2\xe2\xee \xed\xe5 \xf0\xe0\xe1\xee\xf2\xe0\xe5\xf2.\r\n\n'

In [225]:
adc.get_dot_prod_ave(3)

array('B', [0, 0, 0, 0, 0, 0, 0, 0])


0

In [230]:
round(1.2)

1